In [1]:
import json
import deepdoctection as dd

In [2]:
page = dd.load_page("deepdocdetect_output.json")

/Users/jhoetter/deepdoctection/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
scale_x = 819 / 2481 # png width / root level width
scale_y = 1060 / 3508

Categories:
- table
- title
- text
- cell

In [46]:
annotations = {
    "tables": page.get_annotation(category_names="table"),
    "titles": page.get_annotation(category_names="title"),
    "texts": page.get_annotation(category_names="text")
}

In [47]:
for key, list in annotations.items():
    print(key, len(list))

tables 1
titles 2
texts 1


In [48]:
len(page.annotations)

410

In [51]:
table_example = annotations["tables"][0]

In [52]:
title_example = annotations["titles"][0]

In [53]:
text_example = annotations["texts"][0]

In [54]:
with open("deepdocdetect_output.json", "r") as f:
    content = json.load(f)

In [55]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [60]:
table = table_example

In [74]:
def get_table_values(table):
    table_relationships = table.relationships[next(iter(table.relationships))]
    annotation_id_to_value = {}
    for idx, annotation in enumerate(page.get_annotation(annotation_ids=table_relationships)):
        if len(annotation.relationships) > 0:
            value_list = []
            for cell_annotation in page.get_annotation(
                annotation_ids=annotation.relationships[next(iter(annotation.relationships))]
            ):
                value = cell_annotation.sub_categories[next(iter(cell_annotation.sub_categories))].value
                value_list.append(value)
            annotation_id_to_value[annotation.annotation_id] = " ".join(value_list)
    return annotation_id_to_value

In [75]:
def get_text_values(text):
    text_relationships = text.relationships[next(iter(text.relationships))]
    annotation_id_to_value = {}
    for annotation in page.get_annotation(annotation_ids=text_relationships):
        value = annotation.sub_categories[next(iter(annotation.sub_categories))].value
        annotation_id_to_value[annotation.annotation_id] = value
    return annotation_id_to_value

In [76]:
table_html_list = table_example.sub_categories[next(iter(table_example.sub_categories))].value
table_html = "\n".join(table_html_list)
for annotation_id, value in get_table_values(table_example).items():
    table_html = table_html.replace(annotation_id, value)
print(table_html)

<table>
<tr>
<td colspan=2>
INVOICE NUMBER
</td>
<td>
INVOICE DATE
</td>
<td>
PAYMENT DUE DATE | OUR PROJECT NO.
</td>
<td colspan=2>
BALANCE DUE
</td>
</tr>
<tr>
<td colspan=2>
GALT-009
</td>
<td>
Aug 31, 2013
</td>
<td>
Sep 30, 2013 2012-0001
</td>
<td colspan=2>
$11,812.50
</td>
</tr>
<tr>
<td>
</td>
<td>
</td>
<td rowspan=2 colspan=3>
Current Invoice Fee Summary Previously Invoiced Stipulated %phase Amount billed % Value % completed Complete completed
</td>
<td>
Remaining
</td>
</tr>
<tr>
<td>
</td>
<td>
</td>
<td>
Stipulated %phase Amount billed % completed Complete
</td>
</tr>
<tr>
<td colspan=2>
Schematic Design
</td>
<td>
15.63%
</td>
<td>
$80,000.00 100.00% $80,000.00 0.00%
</td>
<td>
$0.00
</td>
<td>
$0.00
</td>
</tr>
<tr>
<td colspan=2>
Design Development
</td>
<td colspan=2>
23.44% $120,000.00 100.00% $120,000.00 0.00%
</td>
<td>
$0.00
</td>
<td>
$0.00
</td>
</tr>
<tr>
<td colspan=2>
Construction Documents
</td>
<td colspan=2>
29.30% 150,000.00 66.00%
</td>
<td>
$9,000.00
<

In [ ]:
img = plt.imread("examples/invoice-1.png")
fig, ax = plt.subplots(1, figsize=(15, 15))

# Display the image
ax.imshow(img)

rows = [{
    'ulx': example.bounding_box.ulx * scale_x,
    'uly': example.bounding_box.uly * scale_y,
    'lrx': example.bounding_box.lrx * scale_x,
    'lry': example.bounding_box.lry* scale_y,
    "content": "wow"
}]
for box in rows:
    print(box['uly'], box['lry'])
    # rect = patches.Rectangle((box['position_x0'], box['position_y0']), box['position_x1']-box['position_x0'], box['position_y1']-box['position_y0'], linewidth=1, edgecolor='r', facecolor='none')
    rect = patches.Rectangle((box['ulx'], box['uly']), box['lrx']-box['ulx'], box['lry']-box['uly'], linewidth=1, edgecolor='r', facecolor='none')
    print(rect)
    ax.add_patch(rect)
    #ax.annotate(box['content'], (box['ulx'], box['uly']), fontsize=8, color='r')

# Show the image
plt.show()